# ДЗ к Уроку 1

## Практическое задание по клиентской аналитике 
1. Залить в свою БД данные по продажам (часть таблицы Orders в csv, исходник здесь https://drive.google.com/drive/folders/1C3HqIJcABblKM2tz8vPGiXTFT7MisrML?usp=sharing) 
2. Проанализировать, какой период данных выгружен 
3. Посчитать кол-во строк, кол-во заказов и кол-во уникальных пользователей, кот совершали заказы. 
4. По годам посчитать средний чек, среднее кол-во заказов на пользователя, сделать вывод , как изменялись это показатели Год от года. 
5. Найти кол-во пользователей, кот покупали в одном году и перестали покупать в следующем. 
6. Найти ID самого активного по кол-ву покупок пользователя.

### 1. Залить в свою БД данные по продажам

In [1]:
import pandas as pd
#import re

In [2]:
df = pd.read_csv('D:/GeekBrains/Database for analytics/orders_20190822.csv', sep=";")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2002804 entries, 0 to 2002803
Data columns (total 4 columns):
id_o       int64
user_id    int64
price      object
o_date     object
dtypes: int64(2), object(2)
memory usage: 61.1+ MB


In [32]:
df['o_date'] = df['o_date'].astype('datetime64[D]')

In [8]:
df['price'] = [x.replace(',', '.') for x in df['price']]
df['price'] = df['price'].astype(float)

In [40]:
df.groupby(df['o_date'].dt.year)

In [36]:
#df['o_date'].dt.year

0          2016
1          2016
2          2016
3          2016
4          2016
5          2016
6          2016
7          2016
8          2016
9          2016
10         2016
11         2016
12         2016
13         2016
14         2016
15         2016
16         2016
17         2016
18         2016
19         2016
20         2016
21         2016
22         2016
23         2016
24         2016
25         2016
26         2016
27         2016
28         2016
29         2016
           ... 
2002774    2017
2002775    2017
2002776    2017
2002777    2017
2002778    2017
2002779    2017
2002780    2017
2002781    2017
2002782    2017
2002783    2017
2002784    2017
2002785    2017
2002786    2017
2002787    2017
2002788    2017
2002789    2017
2002790    2017
2002791    2017
2002792    2017
2002793    2017
2002794    2017
2002795    2017
2002796    2017
2002797    2017
2002798    2017
2002799    2017
2002800    2017
2002801    2017
2002802    2017
2002803    2017
Name: o_date, Length: 20

In [9]:
df.describe()

,id_o,user_id,price
count,2.002804e+06,2.002804e+06,2.002804e+06
mean,3.968491e+06,2.381755e+06,2.268164e+03
std,1.682212e+06,1.633211e+06,3.359283e+03
min,1.234491e+06,0.000000e+00,-1.848000e+02
25%,2.460856e+06,1.108048e+06,7.672000e+02
50%,3.978659e+06,2.028087e+06,1.441300e+03
75%,5.391492e+06,3.679133e+06,2.552900e+03
max,6.945534e+06,5.919156e+06,8.190966e+05


In [10]:
df.head(5)

,id_o,user_id,price,o_date
0,1234491,337544,539.0,01.01.2016
1,1234494,171642,153.3,01.01.2016
2,1234497,260596,55.3,01.01.2016
3,1234498,1105609,752.5,01.01.2016
4,1234500,982696,4410.0,01.01.2016


### 2. Проанализировать, какой период данных выгружен

In [11]:
df['o_date'].min(), df['o_date'].max()

('01.01.2016', '31.12.2017')

### 3. Посчитать кол-во строк, кол-во заказов и кол-во уникальных пользователей, кот совершали заказы.

#### Количество строк

In [12]:
len(df)

2002804

#### Количество заказов

In [13]:
len(df["id_o"].unique())

2002804

#### Количество уникальных пользователей, которые совершали заказы

In [14]:
len(df['user_id'].unique())

1015119

### 4. По годам посчитать средний чек, среднее кол-во заказов на пользователя, сделать вывод , как изменялись эти показатели год от года.

#### По годам посчитать средний чек

In [45]:
mean_price = df.groupby(df['o_date'].dt.year)[['price']].mean().rename(columns={'price': 'mean_price'})

In [46]:
mean_price

,mean_price
o_date,
2016,2095.583717
2017,2398.392923


#### По годам посчитать среднее количество заказов на пользователя

In [99]:
mean_order_per_user = {}#pd.DataFrame()
for year in df['o_date'].dt.year.unique():
    df_year = df.loc[(df['o_date'].dt.year == year)]
    mean_order_per_user[year] = df_year.groupby('user_id').count()[['id_o']].mean()[0]
print(mean_order_per_user)

{2016: 1.9352089006317794, 2017: 1.74296603725183}


#### Сделать вывод , как изменялись эти показатели год от года.

Средний чек возрос, однако среднее количество заказов на одного пользователя снизилось. Скорее всего это означает, что пользователи стали делать обращаться в магазин немного реже, но делать более крупные заказы. 

### 5. Найти кол-во пользователей, кот покупали в одном году и перестали покупать в следующем.

In [101]:
df_2016 = df.loc[(df['o_date'].dt.year == years[0])]
df_2017 = df.loc[(df['o_date'].dt.year == years[1])]

In [102]:
len(df_2016[~df_2016.isin(df_2017)].dropna())

861346

### Найти ID самого активного по кол-ву покупок пользователя.

In [114]:
count_orders = df.groupby('user_id').count()[['id_o']].rename(columns={'id_o': 'count_id_o'})#.max()

In [105]:
most_active_costumer

id_o    3183
dtype: int64

In [ ]:
most_active_costumer = count_orders.loc[(count_orders['id_o'].dt.year == year)]

In [108]:
most_active_costumer = df.groupby('user_id').count()[['id_o']]\
                            .sort_values(by='id_o')
most_active_costumer

,id_o
user_id,
0,1
3882447,1
3882453,1
3882458,1
3882461,1
3882467,1
3882479,1
3882495,1
3882513,1


In [ ]:
most_active_costumer = df.groupby('user_id').count()[['id_o']]\
                            .sort_values(by='id_o').tail(1)
most_active_costumer = most_active_costumer.index[0]
print(f'The most active costumer is {most_active_costumer}.') 

In [115]:
most_active_costumer = df.groupby('user_id').count()[['id_o']]\
                            .sort_values(by='id_o').tail(1)
most_active_costumer = most_active_costumer.index[0]
print(f'The most active costumer is {most_active_costumer}.') 

The most active costumer is 765861.
